<div style="background-color: #4CAF50; color: white; padding: 10px; border-radius: 5px; display: flex; align-items: center;">
    <h1 style="margin: 0 auto; font-size: 30px; font-weight: bold; font-family: Helvetica">การวิเคราะห์ข้อมูลด้วยโครงข่ายประสาทเทียม (Neural Network)</h1>
</div>

<h1 style="font-size: 25px; font-weight: bold; font-family: Helvetica">เครื่องมือที่จะใช้ใน Lab นี้:</h1>

In [1]:
# นำเข้าไลบรารี numpy โดยตั้งชื่อว่า np
import numpy as np
# นำเข้าไลบรารี matplotlib โดยตั้งชื่อว่า plt
import matplotlib.pyplot as plt
# นำเข้าไลบรารี pandas โดยตั้งชื่อว่า pd
import pandas as pd
# นำเข้าไลบรารี seaborn โดยตั้งชื่อว่า sns
import seaborn as sns

<h1 style="font-size: 25px; font-weight: bold; font-family: Helvetica">1. สำรวจและวิเคราะห์ข้อมูลเบื้องต้น (Exploratory Data Analysis: EDA)</h1>

<h1 style="font-size: 18px;">1.1 นำข้อมูลเข้า</h1>

In [ ]:
data = pd.read_csv('drug200_5classes.csv') # อ่านไฟล์ csv ชื่อ 'drug200_5classes.csv' เราจะได้ตารางของข้อมูล (dataframe) เก็บไว้ในตัวแปรชื่อ data
data.head() # แสดงผล dataframe 5 แถวแรก

<h1 style="font-size: 18px;">1.2 สำรวจข้อมูล</h1>

In [ ]:
data.info() # แสดงรายละเอียดของแต่ละคอลัมน์ใน data

In [ ]:
data.describe() # แสดงสถิติเบื้องต้นของแต่ละคอลัมน์ใน dataframe

In [ ]:
# ตรวจสอบข้อมูลที่หายไป
data.isnull().sum()

<h1 style="font-size: 18px;">1.3 เปลี่ยน label จาก Nominal categorical เป็น Ordinal categorical</h1>

In [ ]:
# นำเข้าไลบรารี sklearn.preprocessing เพื่อเรียกใช้ LabelEncoder
from sklearn.preprocessing import LabelEncoder
# สำหรับ y ที่เป็น categorical (เช่น string labels)
label_encoder = LabelEncoder()
# แปลงข้อมูลในคอลัมน์ ที่เป็น String ให้เป็นตัวเลขคลาส
data['Sex'] = label_encoder.fit_transform(data['Sex'])
data['BP'] = label_encoder.fit_transform(data['BP'])
data['Cholesterol'] = label_encoder.fit_transform(data['Cholesterol'])
data['Drug'] = label_encoder.fit_transform(data['Drug'])
data

<h1 style="font-size: 18px;">1.4 ดูความสัมพันธ์ของข้อมูล</h1>

In [ ]:
# สร้างกราฟ corelation ของแต่ละคอลัมน์ใน dataframe
corr_matrix = data.corr()  # เรียกดู correlation ของแต่ละคอลัมน์ใน dataframe
plt.figure(figsize=(10, 8)) # กำหนดขนาดของกราฟ
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=2) # สร้างกราฟ correlation matrix 
plt.title('Correlation Heatmap') # ตั้งชื่อกราฟ
plt.show() # แสดงผล

In [ ]:
data.corr().Drug.sort_values(ascending=False) # แสดงผลคอลัมน์ที่มี correlation กับ 'Class' จากมากไปน้อย

<h1 style="font-size: 18px;">1.6 ดูการกระจายตัวของข้อมูล</h1>

In [ ]:
# แสดงจำนวนของแต่ละคลาสในคอลัมน์ 'Drug'
class_counts = data['Drug'].value_counts()
print(class_counts)

# แสดงการกระจายตัวของคลาสในคอลัมน์ 'Drug'
plt.figure(figsize=(6, 4))
sns.countplot(data=data, x='Drug')
plt.title('Distribution of Drug')
plt.xlabel('Drug')
plt.ylabel('Count')
plt.show()

<h1 style="font-size: 25px; font-weight: bold; font-family: Helvetica">2. ปรับปรุงข้อมูล (Data Preprocessing)</h1>

In [10]:
# นำเข้าไลบรารี sklearn.preprocessing เพื่อเรียกใช้ StandardScaler
from sklearn.preprocessing import StandardScaler

# นำฟังก์ชัน StandardScaler() เก็บไว้ในตัวแปร pre_process
scaler = StandardScaler()

<h1 style="font-size: 18px;">2.1 ทำการ Standardize ข้อมูลส่วน Features ที่ต้องการ</h1>

In [ ]:
data['Age'] = scaler.fit_transform(data[['Age']])
data['Na_to_K'] = scaler.fit_transform(data[['Na_to_K']])
data

<h1 style="font-size: 18px;">2.2 แบ่งข้อมูลเป็นส่วน Features และ Label</h1>

In [12]:
# X เป็น Features
X = data.drop('Drug', axis=1)
# y เป็น Label
y = data['Drug']
# แปลง X ให้เป็น Array
X = np.array(X)
# แปลง y ให้เป็น Array
y = np.array(y)

In [ ]:
print(type(X)) ; print(type(y)) # แสดง type ของ X และ y

<h1 style="font-size: 25px; font-weight: bold; font-family: Helvetica">3. แบ่งส่วนข้อมูล (Train-Test Split)</h1>

In [14]:
# นำเข้าไลบรารี sklearn.model_selection เพื่อเรียกใช้ train_test_split
from sklearn.model_selection import train_test_split

In [15]:
# แบ่งส่วนข้อมูล X เป็น X_train, X_test และ y เป็น y_train, y_test โดยมีอัตราส่วนแบ่งเป็น 0.8:0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)

In [ ]:
print('length of X_train:',len(X_train),', length of X_test:', len(X_test)) # แสดงจำนวนข้อมูลใน X_train และ X_test

In [ ]:
# หรืออีกวิธีหนึ่ง เราสามารถดู shape ของข้อมูลที่เราแบ่งได้ โดยใช้คำสั่ง .shape 
print('shape of X_train:', X_train.shape)   # ดู shape ของ X_train
print('shape of X_test:', X_test.shape)     # ดู shape ของ X_test
print('shape of y_train:', y_train.shape)   # ดู shape ของ y_train
print('shape of y_test:', y_test.shape)     # ดู shape ของ y_test

<h1 style="font-size: 25px; font-weight: bold; font-family: Helvetica">4. การใช้ Neural Networks Model ด้วย Scikit-Learn Library</h1>

In [18]:
 # นำเข้าไลบรารี sklearn.neural_network เพื่อเรียกใช้ MLPClassifier
from sklearn.neural_network import MLPClassifier

<h1 style="font-size: 18px;">4.1 สร้างโมเดล และ กำหนดการ compile ของโมเดล</h1>

In [19]:
# สร้างโมเดล
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32, 16), # กำหนด Input layer, Hidden Layers
    activation='relu',               # Activation function ที่จะใช้
    solver='adam',                   # Optimizer ที่จะใช้
    learning_rate_init=0.001,        # กำหนด learning rate
    alpha=0.0001,                    # คือพารามิเตอร์ L2 penalty (regularization term)
    batch_size=32,                   # สำหรับ SGD Algorithm
    max_iter=50,                     # จำนวนรอบการเทรน
    verbose=True,                    # แสดงผลระหว่างเทรน
    random_state=42                  # ควบคุมการทำผลซ้ำ
)

<h1 style="font-size: 18px;">4.2 เทรนโมเดล</h1>

In [ ]:
# Train the model
mlp.fit(X_train, y_train)

<h1 style="font-size: 18px;">4.3 ใช้โมเดลที่เทรนแล้ว ทำนายข้อมูล unseen data อย่าง X_test</h1>

In [21]:
y_pred_mlp = mlp.predict(X_test)
y_proba = mlp.predict_proba(X_test)

In [ ]:
y_pred_mlp

In [ ]:
y_proba

In [ ]:
y_proba.round(2)

In [ ]:
y_proba = np.argmax(y_proba, axis=1)
y_proba

<h1 style="font-size: 25px; font-weight: bold; font-family: Helvetica">5. การใช้ Neural Networks Model ด้วย Tensorflow-Keras</h1>

In [26]:
from tensorflow.keras.models import Sequential      # นำเข้าไลบรารี tensorflow.keras.models เพื่อเรียกใช้ Sequential
from tensorflow.keras.layers import Dense, Dropout  # นำเข้าไลบรารี tensorflow.keras.layers เพื่อเรียกใช้ Dense, Dropout
from tensorflow.keras.optimizers import Adam        # นำเข้าไลบรารี from tensorflow.keras.optimizers เพื่อเรียกใช้ Adam

<h1 style="font-size: 18px;">5.1 สร้างโมเดล</h1>

In [27]:
# สร้างโมเดล
model = Sequential()

# เพิ่ม Hidden layer แรก หลังจาก Input layer
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
 
# เพิ่ม Hidden layers อื่นๆ
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))

# เพิ่ม layer สุดท้าย เรียกว่า output layer
model.add(Dense(5, activation='softmax'))

<h1 style="font-size: 18px;">5.2 กำหนดการ compile ของโมเดล และเทรนโมเดล</h1>

In [28]:
# กำหนดการ compile ของโมเดล
model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# ดูภาพรวมของโครงสร้างโมเดล
model.summary()

In [ ]:
# เทรนโมเดล
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
plt.figure(figsize=(12, 4))

# แสดงกราฟประวัติค่า accuracy ที่เกิดขึ้นระหว่างการเทรนโมเดล
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# แสดงกราฟประวัติค่า loss ที่เกิดขึ้นระหว่างการเทรนโมเดล
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

<h1 style="font-size: 18px;">5.3 ใช้โมเดลที่เทรนแล้ว ทำนายข้อมูล unseen data อย่าง X_test</h1>

In [ ]:
y_pred = model.predict(X_test) # ให้โมเดลทำนาย Class จาก Features ที่กำหนดให้
print(y_pred)

In [ ]:
y_pred = np.argmax(y_pred, axis=1) # ตั้ง Boundary ในการตัดสินใจว่าเป็นคลาสใด
y_pred

In [ ]:
print('y_pred:', y_pred)
print('y_test:', y_test)

<h1 style="font-size: 25px; font-weight: bold; font-family: Helvetica">6. การวัดผลโมเดล (Model Evaluation)</h1>

ในการวัดผลการทำ Binary-Classification Model เราต้องรู้จัก Confusion Matrix ก่อน

In [35]:
from sklearn.metrics import confusion_matrix  # นำเข้าไลบรารี sklearn.metrics เพื่อเรียกใช้ confusion_matrix

In [ ]:
# ตัวอย่างการสร้าง confusion matrix
conf_matrix_sample = confusion_matrix(y_test, y_pred)

# แสดง confusion matrix โดยใช้ไลบรารี seaborn
plt.figure(figsize=(8,6))
sns.heatmap(conf_matrix_sample, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Class 0", "Class 1", "Class 2", "Class 3", "Class 4"],
            yticklabels=["Class 0", "Class 1", "Class 2", "Class 3", "Class 4"])

plt.title("Confusion Matrix")
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()

<h1 style="font-size: 18px; font-weight: bold;">6.1 Accuracy - ค่าความแม่นยำ</h1>

- ความแม่นยำ เป็นตัววัดประสิทธิภาพที่เข้าใจได้ง่ายที่สุด มันคืออัตราส่วนของการทำนายที่ถูกต้องต่อการทำนายทั้งหมด
- สูตร:
$$\text{Accuracy} = \frac{\text{จำนวนการทำนายที่ถูกต้อง}}{\text{จำนวนการทำนายทั้งหมด}}$$
- หรือ:
$$\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}$$

<h1 style="font-size: 18px; font-weight: bold;">6.2 Precision - ความแม่นยำของการทำนายคลาสบวก</h1>

- Precision คืออัตราส่วนของการทำนายคลาสบวกที่ถูกต้องต่อการทำนายคลาสบวกทั้งหมด:
- สูตร:
$$\text{Precision} = \frac{\text{TP}}{\text{TP + FP}}$$

<h1 style="font-size: 18px; font-weight: bold;">6.3 Recall / Sensitivity - ความครอบคลุม</h1>

- คืออัตราส่วนของการทำนายคลาสบวกที่ถูกต้องต่อคลาสบวกทั้งหมดในข้อมูลจริง:
- สูตร:
$$\text{Recall} = \frac{\text{TP}}{\text{TP + FN}}$$

<h1 style="font-size: 18px; font-weight: bold;">6.4 F1-Score - คะแนน F1 </h1>

- เป็นค่าเฉลี่ยแบบ harmonic ระหว่างความแม่นยำของการทำนายคลาสบวกและความครอบคลุม
- เป็นวิธีที่ดีในการแสดงประสิทธิภาพของโมเดลเมื่อข้อมูลบวกและข้อมูลลบมีจำนวนที่ไม่เท่ากัน:
- สูตร:
$$\text{คะแนน F1} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$$

<h1 style="font-size: 18px; font-weight: bold;">6.5 Matthew Correlation Coefficient (MCC) - ค่าสหสัมพันธ์แมทธิว </h1>

- คือค่าสหสัมพันธ์ระหว่างการทำนายและข้อมูลจริง ค่านี้จะอยู่ระหว่าง -1 และ 1 ซึ่งค่า 1 หมายถึงการทำนายที่ดีที่สุด ค่า -1 หมายถึงการทำนายที่แย่ที่สุด และค่า 0 หมายถึงการทำนายแบบสุ่ม:
- สูตร:
$$\text{MCC} = \frac{\text{TP} \times \text{TN} - \text{FP} \times \text{FN}}{\sqrt{(\text{TP} + \text{FP})(\text{TP} + \text{FN})(\text{TN} + \text{FP})(\text{TN} + \text{FN})}}$$


In [37]:
#นำเข้าไลบรารี sklearn.metrics เพื่อเรียกใช้ accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

In [ ]:
# คำนวณผลของ metrics ต่างๆ
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)
mcc = matthews_corrcoef(y_test, y_pred)

# แสดงผล
print(f"ความแม่นยำ (Accuracy): {accuracy:.4f}")
print(f"ความแม่นยำของการทำนายคลาสบวก (Precision): {precision}")
print(f"ความครอบคลุม (Recall): {recall}")
print(f"คะแนน F1 (F1-Score): {f1}")
print(f"ค่าสหสัมพันธ์แมทธิว (MCC): {mcc:.4f}")

<h1 style="font-size: 18px; font-weight: bold;">6.6 Classification Report</h1>

In [39]:
from sklearn.metrics import classification_report # นำเข้าไลบรารี sklearn.metrics เพื่อเรียกใช้ classification_report

In [ ]:
# คำสั่งเรียกดู classification report
report = classification_report(y_test, y_pred)
# แสดงผล
print(report)